In [1]:
import numpy as np
import pandas as pd
import math
import json
import sys
import time

### data manipulation

In [2]:
def readFiles(filename):
    df = pd.read_csv(datafile, header=None)
    
    # restrictions are in first row
    restr = pd.to_numeric(df.iloc[0])
    
    # drop metadata columns
    df = df.drop([0], axis=0)
    
    return df, restr

In [3]:
def restrictdf(df, restr):
    # remove restricted cols and convert to numeric
    for i, v in enumerate(df.columns):
        if restr[i] < 1:
            df = df.drop(columns=[v])
        else:
            df[v] = pd.to_numeric(df[v], errors='coerce')
            
    # drop unknown values
    df = df.dropna()
    df = df[(df != '?').all(axis=1)]
    return df

In [35]:
# normalizes all columns
def normalizedf(indf):
    df=indf.copy()
    for c in df.columns:
        colMax = df[c].max()
        colMin = df[c].min()
        
        # probably no need to normalize if the values are very small. Might have to adjust the value
#         if colMax < 1:
#             continue
        df[c] = df[c].apply(lambda x: (x - colMin)/(colMax-colMin))
    return df

### helper functions

In [5]:
def euclideanDist(point, pointArray):
    return np.sqrt(np.sum((pointArray - point) ** 2, axis=1))

In [6]:
# pass a ***vectorized*** distance function: dist(point, pointArray)
def calcDistMatrix(df, distFunctionVect):
    # must be fully numeric and normalized df
    dfarray = np.array(df)
    
    distMatrix = []
    for i, d in enumerate(dfarray):
        # performs Euclidean distance on all elements in data (vectorized)
        dists = distFunctionVect(dfarray[i], dfarray)
        distMatrix.append(dists)
    
    return pd.DataFrame(distMatrix)

In [7]:
def calcNeighborhoods(distMatrix, epsilon):    
    # iterating through a dictionary is much faster
    dfdict = distMatrix.to_dict('records')
    
    # Sorry, had to do this in one line, filters each row by epsilon
    # k+1 as index in these datasets starts at 1
    return [[k+1 for (k,v) in row.items() if v <= epsilon] for row in dfdict]

In [8]:
def calcCorePoints(neighborhoods, minpts):
    # > because the point itself should be excluded
    return [i+1 for i, v in enumerate(neighborhoods) if len(v) > minpts]

In [9]:
def densityConnected(df, pntId, neighborhoods, cores, currCluster):
    # visit each unvisited neigh, and their neighbors if core. DFS 
    # update visited, clusterid, and type
    
    if df.at[pntId, "visited"]:
        return
    df.at[pntId, "visited"] = True
    for neigh in neighborhoods[pntId-1]:
        if not df.at[neigh, "visited"]:
            df.at[neigh, "cluster"] = currCluster
            if neigh in cores:
                # continue density connectivity
                df.at[neigh, "type"] = "core"
                densityConnected(df, neigh, neighborhoods, cores, currCluster)
            else:
                df.at[neigh, "visited"] = True
                df.at[neigh, "type"] = "boundary"

### analytical functions

In [42]:
# gets centroid of numeric dataframe (not normalized)4
def calcCentroid(numdf):
    return np.divide(np.sum(np.array(numdf), axis=0),len(numdf))

In [11]:
# takes df ran through dbscan with visited, cluster, and type columns
def analyzeClusters(df, numdf):
    clusters=[]
    for i, c in enumerate(df['cluster'].unique()):
        info = {}
        info["clusterID"] = i
        if c is None:
            pnts = df.loc[df['cluster'].isna()]
            info["type"] = "Noise"
        else:
            pnts = df[df['cluster'] == c]
            info["type"] = "Cluster"
        
        numpnts = numdf.loc[pnts.index]
        info["centroid"] = calcCentroid(numpnts)
        
        info["numPoints"] = len(pnts)
        info["dataPoints"] = pnts
        clusters.append(info)
    return clusters

In [12]:
def printClusterInfo(clusters):
    for clusterInfo in clusters:
        for key in clusterInfo:
            if key == "dataPoints":
                print(f"{key}: \n{clusterInfo[key].to_markdown()}")
            else:
                print(f"{key}: {clusterInfo[key]}")
        print('\n')

### dbscan functions

In [13]:
def dbscan_lite(df, neighborhoods, cores):
    df["visited"] = False
    df["cluster"] = None
    df["type"] = "Noise"
    
    currCluster=0
    for c in cores:
        if not df.at[c, "visited"]:
            df.at[c, "type"] = "core"
            df.at[c, "cluster"] = currCluster 
            densityConnected(df, c, neighborhoods, cores, currCluster)
            currCluster += 1
    
    return df

In [27]:
def dbscan(df, restr, distFunc, epsilon, minpnts):
    numdf = restrictdf(df,restr)
    distMatrix = calcDistMatrix(normalizedf(numdf), euclideanDist)
    neighborhoods = calcNeighborhoods(distMatrix, epsilon)
    cores = calcCorePoints(neighborhoods, minpts)
    
    df = dbscan_lite(df, neighborhoods, cores)
    
    print(numdf)
    
    clusters = analyzeClusters(df, numdf)
    printClusterInfo(clusters)

### running

In [43]:
sys.argv = "dbscan.py ./data/iris.csv 0.2 6".split(" ")
if __name__ == "__main__":
    if len(sys.argv) == 4:
        _, datafile, epsilon, minpts = sys.argv
    else:
        print("Usage: python3 dbscan.py <datafile.csv> <epsilon> <numPoints>")
        exit(1)
        
    minpts = float(minpts)
    epsilon = float(epsilon)
    
    df, restr = readFiles(datafile)
    
    dbscan(df, restr, euclideanDist, epsilon, minpts)

       0    1    2    3
1    5.1  3.5  1.4  0.2
2    4.9  3.0  1.4  0.2
3    4.7  3.2  1.3  0.2
4    4.6  3.1  1.5  0.2
5    5.0  3.6  1.4  0.2
..   ...  ...  ...  ...
146  6.7  3.0  5.2  2.3
147  6.3  2.5  5.0  1.9
148  6.5  3.0  5.2  2.0
149  6.2  3.4  5.4  2.3
150  5.9  3.0  5.1  1.8

[150 rows x 4 columns]
clusterID: 0
type: Cluster
centroid: [5.01632653 3.44081633 1.46734694 0.24285714]
numPoints: 49
dataPoints: 
|    |   0 |   1 |   2 |   3 | 4           | visited   |   cluster | type     |
|---:|----:|----:|----:|----:|:------------|:----------|----------:|:---------|
|  1 | 5.1 | 3.5 | 1.4 | 0.2 | Iris-setosa | True      |         0 | core     |
|  2 | 4.9 | 3   | 1.4 | 0.2 | Iris-setosa | True      |         0 | core     |
|  3 | 4.7 | 3.2 | 1.3 | 0.2 | Iris-setosa | True      |         0 | core     |
|  4 | 4.6 | 3.1 | 1.5 | 0.2 | Iris-setosa | True      |         0 | core     |
|  5 | 5   | 3.6 | 1.4 | 0.2 | Iris-setosa | True      |         0 | core     |
|  6 | 5.4 | 3.9 |